# MONALISA T4.1 - EO indices; Landsat

This notebook allows computing 28 RS indices from Landsat based on the "Indicators_vs_Pressures.xslx" document; enhacing reproducibility and comparability

* **Landsat True Color**
* **Soil indices:** Normalized Difference Soil Index (NDSI); Bare Soil Index (BSI); modified Bare Soil Index (MBI)
* **Soil Salinity indices:**  Soil Salinity Index-1 (SSI1); Soil Salinity Index-2 (SSI-2); Soil Salinity Index-3 (SSI-3); Salinitiy Index (SI); Soil Adjusted Salinity Index (SASI); Normalized Difference Salinity Index (NDSaI)
* **Vegetation indices:** Normalized Difference Vegetation Index (NDVI); Red Edge Position (REP); Green Normalized Difference Vegetation Index (GNDVI); Modified Soil Adjusted Vegetation Index (MSAVI); Modified Soil Adjusted Vegetation Index-2 (MSAVI2); Optimized Soil-Adjusted Vegetation Index (OSAVI), Normalized Difference Red Edge (NDRE); Atmospherically Resistant Vegetation Index (ARVI); Enhanced Vegeation Index (EVI); Enhanced Vegetation Index-2 (EVI2); Fractional Vegetation Cover (FVC)
* **Burned Area indices:** Normalized Burn Ratio-1 (NBR1); Normalized Burn Ratio-2 (NBR2)
* **Drought indices:** Normalized Difference Drought Index (NDDI); Desertification Soil Index (DSI)
* **Water indices:** Normalized Difference Water Index-1 (NDWI1); Normalized Difference Water Index-2 (NDWI2); modified Normalized Difference Water Index (MNDWI)

Author(s): Gregory Giuliani, Audrey Lambiel [University of Geneva]

Version: 1.0 [16.10.2025]


## Initialization

In [70]:
import ee
import geemap

In [71]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…

In [72]:
#Config
startDate = '2023-01-01'
endDate = '2024-01-01'
site_name = 'LosPedroches'
year = '2023'

if year >= '2013':
    ls = "LANDSAT/LC08/C02/T1_L2" #Landsat 8
    sensor = 'oli'
elif year == '2012':
    ls = "LANDSAT/LE07/C02/T1_L2" #Landsat 7
    sensor = 'tm'
else:
    ls = "LANDSAT/LT05/C02/T1_L2" #Landsat 5
    sensor = 'tm'

In [73]:
#AoI using assets
aoi = ee.FeatureCollection('projects/ee-rs2/assets/LosPedroches') #alta_murgia; asterousia; medenine; tayasir; LosPedroches
Map.addLayer(aoi,{},'AOI')
Map.centerObject(aoi, 14)

#### *Landsat - True color*

In [48]:
#Landsat
#Cloud masking
def cloudMask(image):
  cloudShadowBitmask = (1 << 3)
  cloudBitmask = (1 << 5)
  qa = image.select('QA_PIXEL')
  mask = qa.bitwiseAnd(cloudShadowBitmask).eq(0) \
                .And(qa.bitwiseAnd(cloudBitmask).eq(0))
  return image.updateMask(mask)

#Import Landsat image collection
CollectionLS = ee.ImageCollection(ls) \
              .filterBounds(aoi) \
              .filterDate(startDate, endDate) \
              .filter(ee.Filter.calendarRange(1, 12, 'month')) \
              .map(cloudMask)

#RGB visualization parameters
if sensor == 'oli':
  bands = ['SR_B4', 'SR_B3', 'SR_B2']
else:
  bands = ['SR_B3', 'SR_B2', 'SR_B1']

visualizationLS = {
  'bands': bands,
  'min': 7000,
  'max': 14000,
}

#Visualize RGB median
MedianLS = CollectionLS.median().clip(aoi)
Map.addLayer(MedianLS, visualizationLS, 'Landsat | True Color')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=MedianLS, 
  description='RGB_LS_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='RGB_LS_'+site_name+'_'+year,
  folder=site_name,
  region=aoi.geometry(),
  scale=30,
  crs='EPSG:4326',
  maxPixels=1e10
)

## Soil Indices

#### *Landsat - NDSI*

In [ ]:
#Normalized Difference Soil Index
if sensor == 'oli':
  ndsi = MedianLS.normalizedDifference(['SR_B6', 'SR_B3']).rename('NDSI')
else:
  ndsi = MedianLS.normalizedDifference(['SR_B5', 'SR_B2']).rename('NDSI')

# Compute min and max values over the AOI
ndsi_stats = ndsi.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=30,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': ndsi_stats.get('NDSI_min').getInfo(),
    'max': ndsi_stats.get('NDSI_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize NDVI
Map.addLayer(ndsi, vis_params, 'Landsat | NDSI')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=ndsi, 
  description='NDSI_LS_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='NDSI_LS_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=30,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Landsat - BSI*

In [ ]:
#Bare Soil Index
if sensor == 'oli':
  bsi = MedianLS.expression(
        '((SWIR1 + RED) - (NIR + BLUE)) / ((SWIR1+RED) + (NIR +BLUE))', {
            'SWIR1': MedianLS.select('SR_B6'),
            'RED': MedianLS.select('SR_B4'),
            'NIR': MedianLS.select('SR_B5'),
            'BLUE': MedianLS.select('SR_B2')
        }).rename('BSI')
else:
  bsi = MedianLS.expression(
        '((SWIR1 + RED) - (NIR + BLUE)) / ((SWIR1+RED) + (NIR +BLUE))', {
            'SWIR1': MedianLS.select('SR_B5'),
            'RED': MedianLS.select('SR_B3'),
            'NIR': MedianLS.select('SR_B4'),
            'BLUE': MedianLS.select('SR_B1')
        }).rename('BSI')

# Compute min and max values over the AOI
bsi_stats = bsi.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=30,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': bsi_stats.get('BSI_min').getInfo(),
    'max': bsi_stats.get('BSI_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(bsi, vis_params, 'Landsat | BSI')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=bsi, 
  description='BSI_LS_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='BSI_LS_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=30,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Landsat - MBI*

In [ ]:
#modified Bare Soil Index
if sensor == 'oli':
  mbi = MedianLS.expression(
        '((SWIR - NIR) - (RED - BLUE)) / ((SWIR - NIR) + (RED - BLUE))', {
            'SWIR': MedianLS.select('SR_B6'),
            'RED': MedianLS.select('SR_B4'),
            'NIR': MedianLS.select('SR_B5'),
            'BLUE': MedianLS.select('SR_B2')
        }).rename('MBI')
else:
  mbi = MedianLS.expression(
        '((SWIR - NIR) - (RED - BLUE)) / ((SWIR - NIR) + (RED - BLUE))', {
            'SWIR': MedianLS.select('SR_B5'),
            'RED': MedianLS.select('SR_B3'),
            'NIR': MedianLS.select('SR_B4'),
            'BLUE': MedianLS.select('SR_B1')
        }).rename('MBI')
  
# Compute min and max values over the AOI
mbi_stats = mbi.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=30,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': mbi_stats.get('MBI_min').getInfo(),
    'max': mbi_stats.get('MBI_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(mbi, vis_params, 'Landsat | MBI')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=mbi, 
  description='MBI_LS_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='MBI_LS_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=30,
  crs='EPSG:4326',
  maxPixels=1e10
)

## Soil Salinity Indices

#### *Landsat - SSI1*

In [ ]:
#Soil Salinity Index-1
if sensor == 'oli':
  ssi1 = MedianLS.expression(
        'B2/B4', {
            'B2': MedianLS.select('SR_B2'),
            'B4': MedianLS.select('SR_B4'),
        }).rename('SSI1')
else:
  ssi1 = MedianLS.expression(
        'B1/B3', {
            'B1': MedianLS.select('SR_B1'),
            'B3': MedianLS.select('SR_B3'),
        }).rename('SSI1')


# Compute min and max values over the AOI
ssi1_stats = ssi1.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=30,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': ssi1_stats.get('SSI1_min').getInfo(),
    'max': ssi1_stats.get('SSI1_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(ssi1, vis_params, 'Landsat | SSI1')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=ssi1, 
  description='SSI1_LS_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='SSI1_LS_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=30,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Landsat - SSI2*

In [ ]:
#Soil Salinity Index-2
if sensor == 'oli':
    ssi2 = MedianLS.expression(
        '((B6+B7)-(B2+B3))/((B6+B7)+(B2+B3))', {
            'B2': MedianLS.select('SR_B2'),
            'B3': MedianLS.select('SR_B3'),
            'B6': MedianLS.select('SR_B4'),
            'B7': MedianLS.select('SR_B7'),
        }).rename('SSI2')
else:
    ssi2 = MedianLS.expression(
        '((B5+B7)-(B1+B2))/((B5+B7)+(B1+B2))', {
            'B1': MedianLS.select('SR_B1'),
            'B2': MedianLS.select('SR_B2'),
            'B5': MedianLS.select('SR_B5'),
            'B7': MedianLS.select('SR_B7'),
        }).rename('SSI2')

# Compute min and max values over the AOI
ssi2_stats = ssi2.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=30,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': ssi2_stats.get('SSI2_min').getInfo(),
    'max': ssi2_stats.get('SSI2_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(ssi2, vis_params, 'Landsat | SSI2')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=ssi2, 
  description='SSI2_LS_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='SSI2_LS_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=30,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Landsat - SSI3*

In [ ]:
#Soil Salinity Index-3
if sensor == 'oli':
    ssi3 = MedianLS.expression(
        '((B6+B7)-(B3+B4))/((B6+B7)+(B3+B4))', {
            'B3': MedianLS.select('SR_B3'),
            'B4': MedianLS.select('SR_B4'),
            'B6': MedianLS.select('SR_B6'),
            'B7': MedianLS.select('SR_B7'),
        }).rename('SSI3')
else:
    ssi3 = MedianLS.expression(
        '((B5+B7)-(B2+B3))/((B5+B7)+(B2+B3))', {
            'B2': MedianLS.select('SR_B2'),
            'B3': MedianLS.select('SR_B3'),
            'B5': MedianLS.select('SR_B5'),
            'B7': MedianLS.select('SR_B7'),
        }).rename('SSI3')

# Compute min and max values over the AOI
ssi3_stats = ssi3.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=30,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': ssi3_stats.get('SSI3_min').getInfo(),
    'max': ssi3_stats.get('SSI3_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(ssi3, vis_params, 'Landsat | SSI3')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=ssi3, 
  description='SSI3_LS_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='SSI3_LS_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=30,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Landsat - SI*

In [ ]:
#Salinitiy Index
if sensor == 'oli':
    si = MedianLS.expression(
        '(B3+B4+B5)/B2', {
            'B2': MedianLS.select('SR_B2'),
            'B3': MedianLS.select('SR_B3'),
            'B4': MedianLS.select('SR_B4'),
            'B5': MedianLS.select('SR_B5'),
        }).rename('SI')
else:
    si = MedianLS.expression(
        '(B2+B3+B4)/B1', {
            'B1': MedianLS.select('SR_B1'),
            'B2': MedianLS.select('SR_B2'),
            'B3': MedianLS.select('SR_B3'),
            'B4': MedianLS.select('SR_B4'),
        }).rename('SI')

# Compute min and max values over the AOI
si_stats = si.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=30,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': si_stats.get('SI_min').getInfo(),
    'max': si_stats.get('SI_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(si, vis_params, 'Landsat | SI')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=si, 
  description='SI_LS_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='SI_LS_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=30,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Landsat - SASI*

In [ ]:
#Soil Adjusted Salinity Index
if sensor == 'oli':
    sasi = MedianLS.expression(
        '((B6-B5)/(B6+B5+0.5))*(1+0.5)', {
            'B5': MedianLS.select('SR_B5'),
            'B6': MedianLS.select('SR_B6')
        }).rename('SASI')
else:
    sasi = MedianLS.expression(
       '((B5-B4)/(B5+B4+0.5))*(1+0.5)', {
            'B4': MedianLS.select('SR_B4'),
            'B5': MedianLS.select('SR_B5')
        }).rename('SASI')
    
# Compute min and max values over the AOI
sasi_stats = sasi.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=30,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': sasi_stats.get('SASI_min').getInfo(),
    'max': sasi_stats.get('SASI_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(sasi, vis_params, 'Landsat | SASI')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=sasi, 
  description='SASI_LS_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='SASI_LS_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=30,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Landsat - NDSaI*

In [ ]:
#Normalized Difference Salinity Index
if sensor == 'oli':
  ndsai = MedianLS.normalizedDifference(['SR_B4', 'SR_B5']).rename('NDSaI')
else:
  ndsai = MedianLS.normalizedDifference(['SR_B3', 'SR_B4']).rename('NDSaI')

# Compute min and max values over the AOI
ndsai_stats = ndsai.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=30,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': ndsai_stats.get('NDSaI_min').getInfo(),
    'max': ndsai_stats.get('NDSaI_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(ndsai, vis_params, 'Landsat | NDSaI')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=ndsai, 
  description='NDSaI_LS_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='NDSaI_LS_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=30,
  crs='EPSG:4326',
  maxPixels=1e10
)

## Vegetation Indices

#### *Landsat - NDVI*

In [ ]:
#Normalized Difference Vegetation Index
if sensor == 'oli':
  ndvi = MedianLS.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
else:
  ndvi = MedianLS.normalizedDifference(['SR_B4', 'SR_B3']).rename('NDVI')

# Compute min and max values over the AOI
ndvi_stats = ndvi.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=30,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': ndvi_stats.get('NDVI_min').getInfo(),
    'max': ndvi_stats.get('NDVI_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(ndvi, vis_params, 'Landsat | NDVI')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=ndvi, 
  description='NDVI_LS_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='NDVI_LS_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=30,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Landsat - REP*

In [ ]:
#Red Edge Position
if sensor == 'oli':
  rep = MedianLS.expression(
        '705+35*((B4-B3)/(B5-B3))', {
            'B3': MedianLS.select('SR_B3'),
            'B4': MedianLS.select('SR_B4'),
            'B5': MedianLS.select('SR_B5')
        }).rename('REP')
else:
  rep = MedianLS.expression(
        '705+35*((B3-B2)/(B4-B2))', {
            'B2': MedianLS.select('SR_B2'),
            'B3': MedianLS.select('SR_B3'),
            'B4': MedianLS.select('SR_B4')
        }).rename('REP')


# Compute min and max values over the AOI
rep_stats = rep.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=30,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': rep_stats.get('REP_min').getInfo(),
    'max': rep_stats.get('REP_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(rep, vis_params, 'Landsat | REP')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=rep, 
  description='REP_LS_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='REP_LS_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=30,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Landsat - GNDVI*

In [ ]:
#Green Normalized Difference Vegetation Index
if sensor == 'oli':
  gndvi = MedianLS.normalizedDifference(['SR_B5', 'SR_B3']).rename('GNDVI')
else:
  gndvi = MedianLS.normalizedDifference(['SR_B4', 'SR_B2']).rename('GNDVI')

# Compute min and max values over the AOI
gndvi_stats = gndvi.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=30,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': gndvi_stats.get('GNDVI_min').getInfo(),
    'max': gndvi_stats.get('GNDVI_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(gndvi, vis_params, 'Landsat | GNDVI')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=gndvi, 
  description='GNDVI_LS_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='GNDVI_LS_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=30,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Landsat - MSAVI*

In [ ]:
#Modified Soil Adjusted Vegetation Index
if sensor == 'oli':
  msavi = MedianLS.expression(
        '((2 * NIR + 1 - sqrt((2 * NIR + 1)**2- 8 * (NIR - RED)))/2)', {
            'NIR': MedianLS.select('SR_B5'),
            'RED': MedianLS.select('SR_B4')
        }).rename('MSAVI')
else:
  msavi = MedianLS.expression(
        '((2 * NIR + 1 - sqrt((2 * NIR + 1)**2- 8 * (NIR - RED)))/2)', {
            'NIR': MedianLS.select('SR_B4'),
            'RED': MedianLS.select('SR_B3')
        }).rename('MSAVI')

# Compute min and max values over the AOI
msavi_stats = msavi.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=30,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': msavi_stats.get('MSAVI_min').getInfo(),
    'max': msavi_stats.get('MSAVI_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(msavi, vis_params, 'Landsat | MSAVI')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=msavi, 
  description='MSAVI_LS_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='MSAVI_LS_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=30,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Landsat - MSAVI2*

In [ ]:
#Modified Soil Adjusted Vegetation Index 2
if sensor == 'oli':
  msavi2 = MedianLS.expression(
        '0.5*((2 * NIR + 1 - sqrt((2 * NIR + 1)**2- 8 * (NIR - RED)))/2)', {
            'NIR': MedianLS.select('SR_B5'),
            'RED': MedianLS.select('SR_B4')
        }).rename('MSAVI2')
else:
  msavi2 = MedianLS.expression(
        '0.5*((2 * NIR + 1 - sqrt((2 * NIR + 1)**2- 8 * (NIR - RED)))/2)', {
            'NIR': MedianLS.select('SR_B4'),
            'RED': MedianLS.select('SR_B3')
        }).rename('MSAVI2')
  
# Compute min and max values over the AOI
msavi2_stats = msavi2.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=30,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': msavi2_stats.get('MSAVI2_min').getInfo(),
    'max': msavi2_stats.get('MSAVI2_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(msavi2, vis_params, 'Landsat | MSAVI2')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=msavi2, 
  description='MSAVI2_LS_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='MSAVI2_LS_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=30,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Landsat - OSAVI*

In [ ]:
#Optimized Soil-Adjusted Vegetation Index


  osavi = MedianLS.expression(
        '1.16*((NIR-RED)/(NIR+RED+0.16))', {
            'NIR': MedianLS.select('SR_B5'),
            'RED': MedianLS.select('SR_B4')
        }).rename('OSAVI')
else:
  osavi = MedianLS.expression(
        '1.16*((NIR-RED)/(NIR+RED+0.16))', {
            'NIR': MedianLS.select('SR_B4'),
            'RED': MedianLS.select('SR_B3')
        }).rename('OSAVI')
  
# Compute min and max values over the AOI
osavi_stats = osavi.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=30,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': osavi_stats.get('OSAVI_min').getInfo(),
    'max': osavi_stats.get('OSAVI_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(osavi, vis_params, 'Landsat | OSAVI')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=osavi, 
  description='OSAVI_LS_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='OSAVI_LS_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=30,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Landsat - NDRE*

In [ ]:
#Normalized Difference Red Edge
if sensor == 'oli':
    ndre = MedianLS.normalizedDifference(['SR_B5', 'SR_B6']).rename('NDRE')
else:
    ndre = MedianLS.normalizedDifference(['SR_B4', 'SR_B5']).rename('NDRE')

# Compute min and max values over the AOI
ndre_stats = ndre.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=30,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': ndre_stats.get('NDRE_min').getInfo(),
    'max': ndre_stats.get('NDRE_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(ndre, vis_params, 'Landsat | NDRE')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=ndre, 
  description='NDRE_LS_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='NDRE_LS_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=30,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Landsat - ARVI*

In [ ]:
#Atomspherically Resistant Vegetation Index
if sensor == 'oli':
    arvi = MedianLS.expression(
            '(NIR-(2*RED-BLUE))/(NIR+(2*RED-BLUE))', {
                'NIR': MedianLS.select('SR_B5'),
                'RED': MedianLS.select('SR_B4'),
                'BLUE': MedianLS.select('SR_B2')
            }).rename('ARVI')
else:
    arvi = MedianLS.expression(
            '(NIR-(2*RED-BLUE))/(NIR-(2*RED-BLUE))', {
                'NIR': MedianLS.select('SR_B4'),
                'RED': MedianLS.select('SR_B3'),
                'BLUE': MedianLS.select('SR_B1')
            }).rename('ARVI')
# Compute min and max values over the AOI
arvi_stats = arvi.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=30,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': arvi_stats.get('ARVI_min').getInfo(),
    'max': arvi_stats.get('ARVI_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(arvi, vis_params, 'Landsat | ARVI')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=arvi, 
  description='ARVI_LS_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='ARVI_LS_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=30,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Landsat - EVI*

In [ ]:
#Enhanced Vegetation Index (EVI)
if sensor == 'oli':
    evi = MedianLS.expression(
            '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
                'NIR': MedianLS.select('SR_B5'),
                'RED': MedianLS.select('SR_B4'),
                'BLUE': MedianLS.select('SR_B2')
            }).rename('EVI')
else:
    evi = MedianLS.expression(
            '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
                'NIR': MedianLS.select('SR_B4'),
                'RED': MedianLS.select('SR_B3'),
                'BLUE': MedianLS.select('SR_B1')
            }).rename('EVI')
    
# Compute min and max values over the AOI
evi_stats = evi.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=30,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': evi_stats.get('EVI_min').getInfo(),
    'max': evi_stats.get('EVI_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(evi, vis_params, 'Landsat | EVI')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=gndvi, 
  description='EVI_LS_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='EVI_LS_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=30,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Landsat - EVI2*

In [ ]:
#Enhanced Vegetation Index 2 (EVI2)
if sensor == 'oli':
    evi2 = MedianLS.expression(
            '2.5 * ((NIR - RED) / (NIR + 2.4*RED + 1))', {
                'NIR': MedianLS.select('SR_B5'),
                'RED': MedianLS.select('SR_B4')
            }).rename('EVI2')
else:
    evi2 = MedianLS.expression(
            '2.5 * ((NIR - RED) / (NIR + 2.4*RED + 1))', {
                'NIR': MedianLS.select('SR_B4'),
                'RED': MedianLS.select('SR_B3')
            }).rename('EVI2')

# Compute min and max values over the AOI
evi2_stats = evi2.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=30,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': evi2_stats.get('EVI2_min').getInfo(),
    'max': evi2_stats.get('EVI2_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize EVI2
Map.addLayer(evi2, vis_params, 'Landsat | EVI2')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=evi2, 
  description='EVI2_LS_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='EVI2_LS_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=30,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Landsat - FVC*

In [89]:
#Fractional Vegetation Cover (FVC)
ndviStats = ndvi.reduceRegion(**{
  'reducer': ee.Reducer.minMax(),
  'geometry': aoi,
  'scale': 30,
  'maxPixels': 1e9
})
ndviMin = ee.Number(ndviStats.get('NDVI_min'))
ndviMax = ee.Number(ndviStats.get('NDVI_max'))

# Visualization parameters
vis_params = {
    'min': ndviMin,
    'max': ndviMax,
    'palette': ['red', 'white', 'green']
}

#Fractional Vegetation Cover
fvc = ((ndvi.subtract(ndviMin)).divide(ndviMax.subtract(ndviMin))) \
          .pow(ee.Number(2)) \
          .rename('FVC')

Map.addLayer(fvc, vis_params, 'Landsat | FVC')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=fvc, 
  description='FVC_LS_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='FVC_LS_'+site_name+'_'+year,
  folder='vegetation'+site_name,  
  region=aoi.geometry(),
  scale=30,
  crs='EPSG:4326',
  maxPixels=1e10
)

## Burned Areas Indices

#### *Landsat - NBR*

In [ ]:
#Normalized Burn Ratio-1
if sensor == 'oli':
  nbr1 = MedianLS.normalizedDifference(['SR_B5', 'SR_B7']).rename('NBR1')
else:
  nbr1 = MedianLS.normalizedDifference(['SR_B4', 'SR_B7']).rename('NBR1')

# Compute min and max values over the AOI
nbr1_stats = nbr1.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=30,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': nbr1_stats.get('NBR1_min').getInfo(),
    'max': nbr1_stats.get('NBR1_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize NBR
Map.addLayer(nbr1, vis_params, 'Landsat | NBR1')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=nbr1, 
  description='NBR1_LS_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='NBR1_LS_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=30,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Landsat - NBR2*

In [ ]:
#Normalized Burn Ratio-2
if sensor == 'oli':
  nbr2 = MedianLS.normalizedDifference(['SR_B6', 'SR_B7']).rename('NBR2')
else:
  nbr2 = MedianLS.normalizedDifference(['SR_B5', 'SR_B7']).rename('NBR2')

# Compute min and max values over the AOI
nbr2_stats = nbr2.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=30,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': nbr2_stats.get('NBR2_min').getInfo(),
    'max': nbr2_stats.get('NBR2_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(nbr2, vis_params, 'Landsat | NBR2')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=nbr2, 
  description='NBR2_LS_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='NBR2_LS_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=30,
  crs='EPSG:4326',
  maxPixels=1e10
)

## Drought Indices

#### *Landsat - NDDI*

In [ ]:
#Normalized Difference Drought Index
if sensor == 'oli':
  nddi = MedianLS.expression(
        '(((B5-B4)/(B5+B4)) - ((B5-B6)/(B5+B6)))/(((B5-B4)/(B5+B4)) + ((B5-B6)/(B5+B6)))', {
            'B4': MedianLS.select('SR_B4'),
            'B5': MedianLS.select('SR_B5'),
            'B6': MedianLS.select('SR_B6')
        }).rename('NDDI')
else:
  nddi = MedianLS.expression(
        '(((B4-B3)/(B4+B3)) - ((B4-B5)/(B4+B5)))/(((B4-B3)/(B4+B3)) + ((B4-B5)/(B4+B5)))', {
            'B4': MedianLS.select('SR_B4'),
            'B5': MedianLS.select('SR_B5'),
            'B3': MedianLS.select('SR_B3')
        }).rename('NDDI')

# Compute min and max values over the AOI
nddi_stats = nddi.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=30,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': nddi_stats.get('NDDI_min').getInfo(),
    'max': nddi_stats.get('NDDI_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(nddi, vis_params, 'Landsat | NDDI')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=nddi, 
  description='NDDI_LS_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='NDDI_LS_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=30,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Landsat - DSI*
This index is not available for Landsat data

## Water Indices

#### *Landsat - NDWI1*

In [ ]:
#Normalized Difference Water Index-1
if sensor == 'oli':
  ndwi1 = MedianLS.normalizedDifference(['SR_B3', 'SR_B5']).rename('NDWI1')
else:
  ndwi1 = MedianLS.normalizedDifference(['SR_B2', 'SR_B4']).rename('NDWI1')

# Compute min and max values over the AOI
ndwi1_stats = ndwi1.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=30,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': ndwi1_stats.get('NDWI1_min').getInfo(),
    'max': ndwi1_stats.get('NDWI1_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(ndwi1, vis_params, 'Landsat | NDWI1')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=ndwi1, 
  description='NDWI1_LS_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='NDWI1_LS_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=30,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Landsat - NDWI2*

In [ ]:
#Normalized Difference Water Index-2
if sensor == 'oli':
  ndwi2 = MedianLS.normalizedDifference(['SR_B5', 'SR_B6']).rename('NDWI2')
else:
  ndwi2 = MedianLS.normalizedDifference(['SR_B4', 'SR_B5']).rename('NDWI2')

# Compute min and max values over the AOI
ndwi2_stats = ndwi2.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=30,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': ndwi2_stats.get('NDWI2_min').getInfo(),
    'max': ndwi2_stats.get('NDWI2_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(ndwi2, vis_params, 'Landsat | NDWI2')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=ndwi2, 
  description='NDWI2_LS_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='NDWI2_LS_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=30,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Landsat - MNDWI*

In [ ]:
#modified Normalized Difference Water Index
if sensor == 'oli':
  mndwi = MedianLS.normalizedDifference(['SR_B3', 'SR_B6']).rename('MNDWI')
else:
  mndwi = MedianLS.normalizedDifference(['SR_B2', 'SR_B5']).rename('MNDWI')

# Compute min and max values over the AOI
mndwi_stats = mndwi.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=30,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': mndwi_stats.get('MNDWI_min').getInfo(),
    'max': mndwi_stats.get('MNDWI_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(mndwi, vis_params, 'Landsat | MNDWI')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=mndwi, 
  description='MNDWI_LS_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='MNDWI_LS_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=30,
  crs='EPSG:4326',
  maxPixels=1e10
)